In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
import pandas as pd
import nibabel as nib
from datetime import datetime

from mfs_tools.library.clustering_stuff import identify_networks

save_to = Path("/mnt/cache/pfm_python")
ml_base_path = Path("/mnt/cache/pfm_matlab")
python_cifti_path = (
    save_to /
    "sub-ME01_task-rest_concatenated_demeaned_regressed_and_smoothed-2.55_32k_fsLR.dtseries.nii"
)
matlab_cifti_path = (
    ml_base_path /
    "sub-ME01_task-rest_concatenated_demeaned_regressed_and_smoothed-2.55_32k_fsLR.dtseries.nii"
)
surface_files = {
    'lh': Path(
        "/mnt/brunodata/open_data/ds005118/derivatives/sub-ME01/fs_LR/fsaverage_LR32k"
        "/ME01.L.midthickness.32k_fs_LR.surf.gii"
    ),
    'rh': Path(
        "/mnt/brunodata/open_data/ds005118/derivatives/sub-ME01/fs_LR/fsaverage_LR32k"
        "/ME01.R.midthickness.32k_fs_LR.surf.gii"
    ),
}
python_distance_matrix_path = save_to / "dist_complete.npy"
matlab_distance_matrix_path = ml_base_path / "DistanceMatrix.mat"

work_dir = Path("/mnt/cache/pfm_python/naming_work")
work_dir.mkdir(exist_ok=True, parents=True)

for p in (
    save_to, ml_base_path, python_cifti_path, matlab_cifti_path,
    surface_files['lh'], surface_files['rh'],
    python_distance_matrix_path, matlab_distance_matrix_path,
    work_dir,
):
    if not p.exists():
        print(f"File '{str(p)}' does not exist.")

#  5.5 to 5.7GB RAM

In [194]:
from mfs_tools.library.file_stuff import load_lynch_network_priors

priors = load_lynch_network_priors(ml_base_path / "priors.mat")


In [19]:
# Use the matlab-generated file to assure most-valid comparisons while debugging
bold_image = nib.cifti2.Cifti2Image.from_filename(
    matlab_cifti_path
)

In [83]:
# Load the matlab-generated network atlases
row = 5  # indicating the 0.01 threshold, per Lynch tutorial code
all_community_labels = nib.cifti2.Cifti2Image.from_filename(
    ml_base_path / "Bipartite_PhysicalCommunities+SpatialFiltering.dtseries.nii"
).get_fdata()
for i in range(all_community_labels.shape[0]):
    print(f"row {i}. {len(np.unique(all_community_labels[i, :]))} unique labels ")
community_labels = all_community_labels[row, :]


row 0. 8 unique labels 
row 1. 14 unique labels 
row 2. 3 unique labels 
row 3. 4 unique labels 
row 4. 6 unique labels 
row 5. 7 unique labels 
row 6. 9 unique labels 
row 7. 14 unique labels 
row 8. 22 unique labels 


In [ ]:
print(datetime.now())

out_file = "Bipartite_PhysicalCommunities+AlgorithmicLabeling"

identify_networks(
    bold_image,
    community_labels,
    surface_files,
    priors,
    save_to / out_file,
    save_to,
    verbose=True,
)

print(datetime.now())


In [24]:
import numpy as np
from mfs_tools.library.cifti_stuff import get_brain_model_axes
brain_axis = get_brain_model_axes(bold_image)
num_lh_ctx_vertices = np.sum(brain_axis.name == 'CIFTI_STRUCTURE_CORTEX_LEFT')
num_rh_ctx_vertices = np.sum(brain_axis.name == 'CIFTI_STRUCTURE_CORTEX_RIGHT')
print(f"{num_lh_ctx_vertices:,} on the left, "
      f"{num_rh_ctx_vertices:,} on the right, "
      f"{num_lh_ctx_vertices + num_rh_ctx_vertices:,} total")


29,696 on the left, 29,716 on the right, 59,412 total


In [25]:
from mfs_tools.library.utility_stuff import correlate_bold
from mfs_tools.library.cifti_stuff import get_cortical_data, get_cortical_indices

cortical_bold_data = get_cortical_data(bold_image)

# Re-calculate the functional connectivity of the BOLD data
bold_conn = correlate_bold(cortical_bold_data)


Overall, correlation took 186s


In [27]:
# Set the diagonal to zero
print("Before:")
print(bold_conn[:6, :6])
np.fill_diagonal(bold_conn, 0.0)
print("Before:")
print(bold_conn[:6, :6])


[[ 1.         -0.299042   -0.3979341   0.20764178 -0.13085835  0.28747034]
 [-0.299042    1.          0.32043928 -0.13593309 -0.06234431 -0.27903906]
 [-0.3979341   0.32043928  1.         -0.15161821  0.12133919 -0.21502851]
 [ 0.20764178 -0.13593309 -0.15161821  1.         -0.14658588  0.32262427]
 [-0.13085835 -0.06234431  0.12133919 -0.14658588  1.         -0.3850531 ]
 [ 0.28747034 -0.27903906 -0.21502851  0.32262427 -0.3850531   1.        ]]
[[ 0.         -0.299042   -0.3979341   0.20764178 -0.13085835  0.28747034]
 [-0.299042    0.          0.32043928 -0.13593309 -0.06234431 -0.27903906]
 [-0.3979341   0.32043928  0.         -0.15161821  0.12133919 -0.21502851]
 [ 0.20764178 -0.13593309 -0.15161821  0.         -0.14658588  0.32262427]
 [-0.13085835 -0.06234431  0.12133919 -0.14658588  0.         -0.3850531 ]
 [ 0.28747034 -0.27903906 -0.21502851  0.32262427 -0.3850531   0.        ]]


In [28]:
# Remove any NaN values
num_nans_before = np.sum(np.isnan(bold_conn))
bold_conn[np.isnan(bold_conn)] = 0.0
num_nans_after = np.sum(np.isnan(bold_conn))
print(f"Removed {num_nans_before - num_nans_after:,} of {num_nans_before} NaNs")


Removed 0 of 0 NaNs


In [84]:
# Explore the infomap-generated communities
unique_community_labels = [
    int(lbl) for lbl in np.unique(community_labels)
    if int(lbl) != 0
]
print(f"Community labels: {unique_community_labels}")


Community labels: [1, 2, 3, 4, 5, 6]


In [37]:
ctx_idx = get_cortical_indices(bold_image)

In [85]:
new_conn = np.zeros(
    (num_lh_ctx_vertices + num_rh_ctx_vertices,
     len(unique_community_labels)),
    dtype=np.float32
)


In [86]:
for i, lbl in enumerate(unique_community_labels):
    print(f"Averaging label {lbl} connectivity at {datetime.now()}...")
    new_conn[:, i] = np.mean(
        bold_conn[:, community_labels[ctx_idx] == lbl],
        axis=1
    )


Averaging label 1 connectivity at 2025-03-02 22:14:27.197492...
Averaging label 2 connectivity at 2025-03-02 22:15:03.889744...
Averaging label 3 connectivity at 2025-03-02 22:15:18.306676...
Averaging label 4 connectivity at 2025-03-02 22:15:29.674022...
Averaging label 5 connectivity at 2025-03-02 22:15:39.025542...
Averaging label 6 connectivity at 2025-03-02 22:15:45.543202...


In [89]:
# From a [59412 x 6] new_conn and a [59412 x 20] prior_fc,
# We'd like an [6 x 20] connectivity matrix
# corrcoef is row-wise, so we transpose the matrices.
rmat = np.corrcoef(new_conn.T, priors.fc.T)[:new_conn.shape[1], new_conn.shape[1]:]
np.save(save_to / f"functional_sim_{row:02d}.npy", rmat)


In [92]:
# Prior spatial is a 59412 x 20 probability map of how likely
# each cortical vertex is to be part of that label.
dmat = np.zeros(rmat.shape)
for i, lbl in enumerate(unique_community_labels):
    for j in range(priors.num_networks):
        dmat[i, j] = np.mean(priors.spatial[community_labels[ctx_idx] == lbl, j])
dmat[np.isnan(dmat)] = 0.0

np.save(save_to / f"spatial_sim_{row:02d}.npy", dmat)


In [71]:
# Load matlab probability matrices for testing and comparing

ml_uci_func = pd.read_csv(ml_base_path / "uCiRho.csv", header=None).values
ml_uci_spat = pd.read_csv(ml_base_path / "uCiSpa.csv", header=None).values


In [93]:
from mfs_tools.library.utility_stuff import compare_mats

compare_mats(rmat, ml_uci_func, tolerance=0.0001, verbose=True)

compare_mats(dmat, ml_uci_spat, tolerance=0.0001, verbose=True)


  The matrices 'a' and 'b' are equal, with tolerance of 0.0001.
  Mem before 16,591.3MB; Mem after 16,591.3MB; delta 0.0
  The matrices 'a' and 'b' are equal, with tolerance of 0.0001.
  Mem before 16,591.3MB; Mem after 16,591.3MB; delta 0.0


True

In [164]:
from mfs_tools.library.similarity_scores import SimilarityScores

real_v_atlas_functional = ml_uci_func
real_v_atlas_spatial = ml_uci_spat
num_communities = len(unique_community_labels)
s = SimilarityScores(num_communities)

for com_idx in range(num_communities):
    prob_combo = real_v_atlas_functional[com_idx, :] * real_v_atlas_spatial[com_idx, :]
    sorted_indices = np.argsort(prob_combo)[::-1]

    s.community[com_idx] = unique_community_labels[com_idx]
    s.r[com_idx] = priors.labels.loc[sorted_indices[0], 'r']
    s.g[com_idx] = priors.labels.loc[sorted_indices[0], 'g']
    s.b[com_idx] = priors.labels.loc[sorted_indices[0], 'b']
    s.network[com_idx] = priors.labels.loc[sorted_indices[0], 'label']
    s.func_conn[com_idx] = real_v_atlas_functional[com_idx, sorted_indices[0]]
    s.spatial_score[com_idx] = real_v_atlas_spatial[com_idx, sorted_indices[0]]
    delta_first_second = prob_combo[sorted_indices[0]] - prob_combo[sorted_indices[1]]
    s.confidence[com_idx] = delta_first_second / prob_combo[sorted_indices[1]]

    # These are offset to store the next-best choices, sequentially
    # My #1 matches matlab #1, and my #19 matches matlab #19. but my #0 is unnecessary?
    for net_idx in range(priors.num_networks - 1):
        s.alt_networks[net_idx][(com_idx,0)] = priors.labels.loc[
            sorted_indices[net_idx + 1], 'label'
        ]
        s.alt_func_sims[net_idx][com_idx] = real_v_atlas_functional[
            com_idx, sorted_indices[net_idx + 1]
        ]
        s.alt_spatial_scores[net_idx][com_idx] = real_v_atlas_spatial[
            com_idx, sorted_indices[net_idx + 1]
        ]

s.report()
s.to_excel(save_to / "similarity_scores.xlsx")


Community: [1 2 3 4 5 6]
Network: [0: CinguloOpercular/Action-mode, 1: Default_Parietal, 2: Default_Parietal, 3: Visual_Lateral, 4: Frontoparietal, 5: DorsalAttention]
Network Manual Decision: []
R: [0.2745,0.9176,0.9176,0.1059,0.9961,0.3882]
G: [0.0275,0.2000,0.2000,0.0078,1.0000,0.8392]
B: [0.5765,0.1373,0.1373,0.5725,0.3294,0.2471]
FC_Similarity: [0.8024,0.5419,0.6028,0.8329,0.7893,0.7821]
Spatial_Score: [0.3011,0.1825,0.2071,0.4086,0.3533,0.3394]
Confidence: [2.2359,0.1774,0.2102,1.6257,1.2526,2.4579]


In [158]:
# Write out a network label/color file for wb_view
with open(save_to / "label_list.txt", "w") as f:
    for net_idx, row in priors.labels.iterrows():
        f.write(row.label)
        f.write(f"\n{row.id} {int(row.r * 255)} {int(row.g * 255)} {int(row.b * 255)} 255\n")


In [212]:
# Alternative to below code block,
from mfs_tools.library.clustering_stuff import map_community_labels

best_guess_community_labels = map_community_labels(
    all_community_labels[5, :],
    s.network,  # corresponds only to row==5 above
    priors.labels,
    verbose=True,
)
print(f"First {10} columns of {best_guess_community_labels.shape}-shaped array:")
print(best_guess_community_labels[:, :10])


Community #0: 29,045 loci get id 14 (CinguloOpercular/Action-mode).
Community #1: 16,696 loci get id 1 (Default_Parietal).
Community #2: 12,612 loci get id 1 (Default_Parietal).
Community #3: 10,010 loci get id 5 (Visual_Lateral).
Community #4: 8,904 loci get id 9 (Frontoparietal).
Community #5: 4,140 loci get id 10 (DorsalAttention).
First 10 columns of (6, 85059)-shaped array:
[[ 0 14 14  0  0  0  0  0  0 14]
 [ 1  0  0  1  0  1  1  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  5  0  0  5  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]]


In [216]:
# Write Cifti around output_data
# Cifti likes to have [t x anat] or [label x anat] data; the output_data match
# The Cifti LabelAxis needs 'num_rows' (output_data.shape[0]) dicts,
# each with 22 keys (zero + 21 labels with colors)
labels = list()
for i in range(best_guess_community_labels.shape[0]):
    _com_id = [_ for _ in np.unique(best_guess_community_labels[i, :]) if _ != 0][0]
    labels.append(str(
        priors.labels.loc[pd.Index(priors.labels.id).get_loc(_com_id), 'label']
    ))
bg_label_axis = nib.cifti2.LabelAxis(
    labels, [priors.cifti_labels for _ in labels],
)
series_axis = nib.cifti2.SeriesAxis(
    0, 1, best_guess_community_labels.shape[0],
)
output_img = nib.cifti2.Cifti2Image(
    best_guess_community_labels, (series_axis, get_brain_model_axes(bold_image))
)
output_img.update_headers()
output_img.to_filename(save_to / f"tmp.dtseries.nii")

output_lbl_img = nib.cifti2.Cifti2Image(
    best_guess_community_labels, (bg_label_axis, get_brain_model_axes(bold_image))
)
output_lbl_img.update_headers()
output_lbl_img.to_filename(save_to / f"best_guess_5.dlabel.nii")

# Don't bother with letting wb_command build this.
# Mine is better with labels for each density image.

In [219]:
# Build one atlas with all labels for density 0.002 (same as row 5)
atlas_lbl_axis = nib.cifti2.LabelAxis(
    ['dens 0.002', ], [priors.cifti_labels, ],
)
atlas_lbl_img = nib.cifti2.Cifti2Image(
    np.sum(best_guess_community_labels, axis=0).reshape(1, -1),
    (atlas_lbl_axis, get_brain_model_axes(bold_image))
)
atlas_lbl_img.update_headers()
atlas_lbl_img.to_filename(save_to / f"best_guess_5_atlas.dlabel.nii")


In [180]:
import subprocess
from mfs_tools.library.file_stuff import find_wb_command_path

wb_command = find_wb_command_path()
out_file = "bipartite_someshit"
cmd_lbl = [
    wb_command, '-cifti-label-import',
    str(save_to / "tmp.dtseries.nii"),
    str(save_to / "label_list.txt"),
    str(save_to / f"{out_file}.dlabel.nii"),
    "-discard-others",
]
cmd_lh = [
    wb_command, '-cifti-label-to-border',
    str(save_to / f"{out_file}.dlabel.nii"),
    "-border", surface_files['lh'],
    str(save_to / f"{out_file}.L.border"),
]
cmd_rh = [
    wb_command, '-cifti-label-to-border',
    str(save_to / f"{out_file}.dlabel.nii"),
    "-border", surface_files['rh'],
    str(save_to / f"{out_file}.R.border"),
]

proc_label = subprocess.run(cmd_lbl)  # no longer necessary
proc_lh_border = subprocess.run(cmd_lh)
proc_rh_border = subprocess.run(cmd_rh)


In [189]:
from mfs_tools.library.cifti_stuff import get_label_axes

# We don't need to do this, but I'd like to inspect the wb_command-generated
# dlabel file. Could I have just done this myself?
wb_label_img = nib.cifti2.Cifti2Image.from_filename(
    save_to / f"{out_file}.dlabel.nii"
)
wb_label_axis = get_label_axes(wb_label_img)


In [201]:
# Matlab lines 189-193: Save functional connectivity for each density?
output_data_2 = np.zeros(
    (len(unique_community_labels), len(brain_axis)),
    dtype=np.float32
)
output_data_2[:, ctx_idx] = new_conn.T
scalar_axis = nib.cifti2.ScalarAxis(
    ['den 0.01', ] + ['empty' for _ in range(len(unique_community_labels) - 1)],
)
output_img_2 = nib.cifti2.Cifti2Image(
    output_data_2, (scalar_axis, brain_axis)
)
output_img_2.update_headers()
output_img_2.to_filename(save_to / f"someshit_FC_WholeBrain_but_really_just_cortex.dtseries.nii")


In [191]:
# Matlab lines 195-205: Save network labels for all communities.
output_data_3 = np.zeros(
    (len(unique_community_labels), len(brain_axis)),
    dtype=np.float32
)
for i, community in unique_community_labels:
    output_data_3[i, all_community_labels == community] = \
        pd.Index(priors.labels.label).get_loc(community) + 1

scalar_axis = nib.cifti2.ScalarAxis(
    ['den 0.01', ] + ['empty' for _ in range(len(unique_community_labels) - 1)],
)
output_img_3 = nib.cifti2.Cifti2Image(
    output_data_3, (scalar_axis, brain_axis)
)
output_img_3.update_headers()
output_img_3.to_filename(save_to / f"another_tmp.dtseries.nii")



85059

In [ ]:
out_file = "bipartite_someshit2"
cmd_lbl2 = [
    wb_command, '-cifti-label-import',
    str(save_to / "another_tmp.dtseries.nii"),
    str(save_to / "label_list.txt"),
    str(save_to / f"{out_file}.dlabel.nii"),
    "-discard-others",
]
proc_label2 = subprocess.run(cmd_lbl2)  # no longer necessary


In [230]:
# Matlab Lines 213-241
# Functional Connectivity Strength between i and ii
final_fc = np.zeros((len(unique_community_labels), all_community_labels.shape[1]),
                    dtype=np.float32)
fc = np.zeros(
    (len(unique_community_labels), len(unique_community_labels)),
    dtype=np.float32)
ci = np.zeros(len(unique_community_labels), dtype=np.uint8)
for i in range(len(unique_community_labels)):
    for j in range(len(unique_community_labels)):
        row_mask = community_labels[ctx_idx] == unique_community_labels[i]
        col_mask = community_labels[ctx_idx] == unique_community_labels[j]
        tmp = bold_conn[row_mask, :][:, col_mask]
        fc[i, j] = np.mean(tmp[~np.isnan(tmp)])
        final_fc[i, community_labels==unique_community_labels[j]] = fc[i, j]

    ci[i] = priors.labels.loc[
        pd.Index(priors.labels.label).get_loc(s.network[i]),
        'id'
    ]


In [232]:
fc_between_infomap_communities_img = nib.cifti2.Cifti2Image(
    final_fc, (series_axis, brain_axis)
)
fc_between_infomap_communities_img.update_headers()
fc_between_infomap_communities_img.to_filename(
    save_to / f"fc_between_infomap_communities.dtseries.nii"
)


Index(['Default_Parietal', 'Default_Anterolateral', 'Default_Dorsolateral',
       'Default_Retrosplenial', 'Visual_Lateral',
       'Visual_Dorsal/VentralStream', 'Visual_V5', 'Visual_V1',
       'Frontoparietal', 'DorsalAttention', 'Premotor/DorsalAttentionII',
       'Language', 'Salience', 'CinguloOpercular/Action-mode',
       'MedialParietal', 'Somatomotor_Hand', 'Somatomotor_Face',
       'Somatomotor_Foot', 'Auditory', 'SomatoCognitiveAction', 'Noise'],
      dtype='object', name='label')